In [1]:
import pandas as pd
import matplotlib
import numpy as np
import pprint as p
import re
from geopy.distance import geodesic as GD


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
child = pd.read_csv('./data/childcare_centers.csv', low_memory=False)
address = pd.read_csv('./data/toronto_addresses.csv', low_memory=False)

In [4]:
child.loc[child['LOC_NAME'].str.contains('Calico Saints Child Care Centre')]

,_id,LOC_ID,LOC_NAME,AUSPICE,ADDRESS,PCODE,ward,PHONE,bldg_type,BLDGNAME,IGSPACE,TGSPACE,PGSPACE,KGSPACE,SGSPACE,TOTSPACE,subsidy,run_date,geometry
110,111,1373,Calico Saints Child Care Centre,Non Profit Agency,35 CALICO DR,M3L 1V5,6,(416) 741-8499,Public Elementary School,Calico Public School,10,15,24,65,60,174,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.50359204..."


In [19]:
address.loc[address['full_addr'].str.contains('37 Castle Knock') ]

,Unnamed: 0,latitude,longitude,source_id,id,group_id,street_no,street,str_name,str_type,str_dir,unit,city,postal_code,full_addr,city_pcs,str_name_pcs,str_type_pcs,str_dir_pcs,csduid,csdname,pruid,provider
81675,81675,43.70517,-79.41182,NaN,705f1ea5785c2ae59e39,3468697,37,Castle Knock Rd,NaN,NaN,NaN,NaN,former Toronto,NaN,37 Castle Knock Rd,FORMER TORONTO,CASTLE KNOCK,RD,NaN,3520005,Toronto,35,Toronto


In [5]:
def pull_geometry(series):
    geometry = eval(series) # Read geometry value as dictionary
    coords = geometry['coordinates'] # Obtain coordinates from geometry value
    return {'longitude': coords[0], 'latitude': coords[1]} # Return longitude and latitude



In [6]:
def add_long_lat(df):
    geometry_dicts = df.geometry.map(pull_geometry).to_list() # Pull geometry from every value and set to list
    geometry_df = pd.DataFrame(geometry_dicts, index=df.index) # Convert list to dataframe
    df = pd.concat([df, geometry_df], axis=1) # Merge both dataframes
    return df

In [7]:
def get_distance(p1, p2):
    distance = GD(p1, p2).km
    return distance

In [8]:
def get_coord(df, index, return_dict=False):
    if return_dict:
        return {'longitude': df['longitude'][index], 'latitude': df['latitude'][index]}
    else:
        return (df['longitude'][index], df['latitude'][index])

In [9]:
child = add_long_lat(child)

In [10]:
get_distance(get_coord(child, 1, return_dict=False), get_coord(child, 50, return_dict=False))

7.214424479364431

In [11]:
child

,_id,LOC_ID,LOC_NAME,AUSPICE,ADDRESS,PCODE,ward,PHONE,bldg_type,BLDGNAME,IGSPACE,TGSPACE,PGSPACE,KGSPACE,SGSPACE,TOTSPACE,subsidy,run_date,geometry,longitude,latitude
0,1,1013,Lakeshore Community Child Care Centre,Non Profit Agency,101 SEVENTH ST,M8V 3B5,3,(416) 394-7601,Public Elementary School,Seventh Street Public School,0,20,32,52,60,164,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.50419384...",-79.504194,43.599924
1,2,1014,Alternative Primary School Parent Group,Non Profit Agency,1100 SPADINA RD,M5N 2M6,8,(416) 322-5385,Public Elementary School,North Preparatory Public School,0,0,12,26,45,83,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.42037489...",-79.420375,43.705766
2,3,1015,Cardinal Leger Child Care Centre (Scarborough),Non Profit Agency,600 MORRISH RD,M1C 4Y1,25,(416) 287-0578,Catholic Elementary School,Cardinal Leger Catholic School,0,10,16,26,50,102,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.17546162...",-79.175462,43.793288
3,4,1016,George Brown - Richmond Adelaide Childcare Centre,Non Profit Agency,130 ADELAIDE ST W,M5H 3P5,10,(416) 415-2453,Other,NaN,10,15,40,0,0,65,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.38381242...",-79.383812,43.649691
4,5,1017,Woodland Nursery School (Warden Woods Communit...,Non Profit Agency,1 FIRVALLEY CRT,M1L 1N8,20,(416) 694-1138x163,High Rise Apartment,NaN,0,10,16,0,0,26,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.28102862...",-79.281029,43.702952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,1043,14335,Atelier Kids Inc.,Commercial Agency,21 QUEBEC AVE,M6P 2T3,4,(416) 419-3864,Other,NaN,0,0,20,24,0,44,N,13JUN22,"{'type': 'Point', 'coordinates': (-79.46683524...",-79.466835,43.653723
1043,1044,14336,Westside Montessori School,Commercial Agency,57 SYLVAN AVE,M6H 1G4,9,(647) 400-6521,Other,NaN,0,12,23,0,0,35,N,13JUN22,"{'type': 'Point', 'coordinates': (-79.43241782...",-79.432418,43.654292
1044,1045,14338,The Neighbourhood Group Harbourfront Child Care,Non Profit Agency,650 QUEENS QUAY W,M5V 3N2,10,(647) 206-4774,High Rise Apartment,NaN,16,20,0,0,0,36,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.39883913...",-79.398839,43.635702
1045,1046,14339,The Neighbourhood Group Waterfront Child Care ...,Non Profit Agency,635 QUEENS QUAY W,M5V 3G3,10,(416) 925-2103x1756,Other,NaN,0,0,40,52,60,152,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.39718759...",-79.397188,43.635749


In [13]:
child.to_csv('./data/childcare_centers_geometry.csv')